In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tp2datostrain/train.csv
/kaggle/input/tp2datostrain/test.csv


In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from category_encoders import CountEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [3]:
import pandas as pd
test = pd.read_csv("../input/tp2datostrain/test.csv")
train = pd.read_csv("../input/tp2datostrain/train.csv")

In [4]:
dfCheto = pd.concat([train,test], sort=False)
dfCheto['precio'] = SimpleImputer(strategy='median').fit_transform(dfCheto[['precio']])
df = dfCheto.copy()

In [5]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [6]:
df.provincia.isna().sum()

197

In [7]:
aux2 = dfCheto[['ciudad', 'provincia']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby(['provincia']).agg(lambda x:x.provincia.value_counts().index[0]).reset_index().set_index('ciudad')
dicc_ciudadPorPcia = aux2.T.to_dict('records').copy()[0]
dicc_ciudadPorPcia['CUALQUIERA'] = 'CUALQUIERCIUDAD'
df['ciudad'] =df['ciudad'].fillna('CUALQUIERA')
df['provincia'] = df['provincia'].fillna(df['ciudad'].map(dicc_ciudadPorPcia))

In [8]:
df.provincia.fillna('provinciaVacia',inplace=True)

In [9]:
df['provincia'].isna().sum()

0

In [10]:
aux2 = dfCheto[['ciudad', 'provincia']].copy()
aux2 = aux2.groupby(['provincia']).agg(lambda x:x.provincia.value_counts().index[0]).reset_index().set_index('ciudad')
aux2.head()
dicc_ciudadMasFrecuentePorPcia = aux2.T.to_dict('records').copy()[0]
dicc_invertido = dict(map(reversed, dicc_ciudadMasFrecuentePorPcia.items()))
df.ciudad.fillna(df.provincia.map(dicc_invertido),inplace=True);
df.ciudad.isna().sum()

0

In [11]:
df.ciudad.fillna('ciudadVacia',inplace=True)

In [12]:
df.tipodepropiedad.fillna('tipoVacio',inplace=True)
'''
auxmetros = df[['tipodepropiedad','metrostotales']].copy()
auxmetros = auxmetros.groupby('tipodepropiedad')['metrostotales'].mean().sort_values().reset_index()
lista = sorted(auxmetros.metrostotales.tolist())
lista_2 = [0]*len(lista)
for i,numero in enumerate(lista):
    if i == 0: lista_2[i] = (0,numero)
    else:
        lista_2[i] = (lista[i-1],lista[i])
rangoMetros = pd.Series(lista_2).sort_values()
auxmetros['rangoMetros'] = rangoMetros
auxmetros = auxmetros.set_index('tipodepropiedad').drop(columns='metrostotales')
dicc_tipoDeProp = auxmetros.T.to_dict('records').copy()[0]
def mapearMetros(metrostotales,dicc_tipoDeProp):
    for clave in dicc_tipoDeProp:
        if metrostotales >= dicc_tipoDeProp[clave][0] and metrostotales < dicc_tipoDeProp[clave][1]: return clave
    return 'Casa'
df.tipodepropiedad.fillna(df.metrostotales.map(lambda x: mapearMetros(x,dicc_tipoDeProp)),inplace=True);
df.tipodepropiedad.isna().sum()
'''

"\nauxmetros = df[['tipodepropiedad','metrostotales']].copy()\nauxmetros = auxmetros.groupby('tipodepropiedad')['metrostotales'].mean().sort_values().reset_index()\nlista = sorted(auxmetros.metrostotales.tolist())\nlista_2 = [0]*len(lista)\nfor i,numero in enumerate(lista):\n    if i == 0: lista_2[i] = (0,numero)\n    else:\n        lista_2[i] = (lista[i-1],lista[i])\nrangoMetros = pd.Series(lista_2).sort_values()\nauxmetros['rangoMetros'] = rangoMetros\nauxmetros = auxmetros.set_index('tipodepropiedad').drop(columns='metrostotales')\ndicc_tipoDeProp = auxmetros.T.to_dict('records').copy()[0]\ndef mapearMetros(metrostotales,dicc_tipoDeProp):\n    for clave in dicc_tipoDeProp:\n        if metrostotales >= dicc_tipoDeProp[clave][0] and metrostotales < dicc_tipoDeProp[clave][1]: return clave\n    return 'Casa'\ndf.tipodepropiedad.fillna(df.metrostotales.map(lambda x: mapearMetros(x,dicc_tipoDeProp)),inplace=True);\ndf.tipodepropiedad.isna().sum()\n"

In [13]:
aux2 = dfCheto[['idzona', 'ciudad']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby('ciudad').agg({'idzona':'median'})
ids = aux2.T.to_dict('records').copy()
ids = ids[0]
#df.idzona.fillna(df.ciudad.map(ids), inplace = True)

In [14]:
df['idzona'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['idzona']])

In [15]:
aux = dfCheto[['lat', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lat':'mean'})
lats = aux.T.to_dict('records').copy()
lats = lats[0]
#df.lat.fillna(df.ciudad.map(lats), inplace = True)

In [16]:
df['lat'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lat']])

In [17]:
aux = dfCheto[['lng', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lng':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
#df.lng.fillna(df.ciudad.map(lngs), inplace = True)

In [18]:
df['lng'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lng']])

In [19]:
aux = dfCheto[['tipodepropiedad', 'habitaciones']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'habitaciones':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
#df.habitaciones.fillna(df.tipodepropiedad.map(habts), inplace = True)

In [20]:
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

In [21]:
aux = dfCheto[['tipodepropiedad', 'garages']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'garages':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
#df.garages.fillna(df.tipodepropiedad.map(habts), inplace = True)

In [22]:
df.garages.fillna(0, inplace=True)

In [23]:
df.drop(columns=['direccion'], inplace=True)

In [24]:
aux = dfCheto[['antiguedad', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'antiguedad':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
#df.antiguedad.fillna(df.ciudad.map(lngs), inplace = True)

In [25]:
df['antiguedad'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['antiguedad']])

In [26]:
aux = dfCheto[['banos', 'tipodepropiedad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'banos':'median'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
#df.banos.fillna(df.tipodepropiedad.map(lngs), inplace = True)

In [27]:
df.banos.fillna(1, inplace=True)

In [28]:
df.titulo.fillna('', inplace=True)
df.descripcion.fillna('', inplace=True)
df['texto'] = df.titulo + ' ' + df.descripcion
df['texto'] = df.texto.str.len()
df.drop(columns = ['descripcion', 'titulo'], inplace = True)

In [29]:
aux = df.copy()

In [30]:
aux.metrostotales.fillna(0, inplace=True)
aux.metroscubiertos.fillna(0, inplace =True)
aux['metrostotalesCorregidos'] = aux[['metrostotales', 'metroscubiertos']].max(axis=1)
aux['metroscubiertosCorregidos'] = aux[['metrostotales', 'metroscubiertos']].min(axis=1)
df['metrostotales'] = aux['metrostotalesCorregidos']
df['metroscubiertos'] = aux['metroscubiertosCorregidos']

In [31]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['dia'] = df.fecha.dt.day
df['mes'] = df.fecha.dt.month
df['anio'] = df.fecha.dt.year
df.drop(columns=['fecha'], inplace=True)

In [32]:
df['publicacionesPorCiudad'] = CountEncoder().fit_transform(df['ciudad'])

In [33]:
df['publicacionesPorProvincia'] = CountEncoder().fit_transform(df['provincia'])

In [34]:
df['publicacionesPorTipo'] = CountEncoder().fit_transform(df['tipodepropiedad'])

In [35]:
df[['tipodepropiedad','provincia','ciudad']].isna().sum()

tipodepropiedad    0
provincia          0
ciudad             0
dtype: int64

In [36]:
transformador_categorias = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

transfomador_numero = Pipeline(steps=[('a', SimpleImputer(strategy = 'median'))])

In [37]:
df.columns

Index(['id', 'tipodepropiedad', 'ciudad', 'provincia', 'antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'precio', 'texto',
       'dia', 'mes', 'anio', 'publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo'],
      dtype='object')

In [38]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['id', 'antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'texto',
       'dia', 'mes', 'anio', 'publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo']),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [39]:

model = xgb.XGBRegressor(learning_rate= 0.1,n_estimators= 2000,scale_pos_weight= 2,max_depth= 5,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.75,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [40]:
entrenamiento = df.head(240000).copy()

In [41]:
prueba = df.tail(60000).copy()

In [42]:
prueba.drop(columns=['precio'], inplace=True)

In [43]:
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()
#train_x, test_x, train_y, test_y = train_test_split(train_x, train_y,train_size=0.8, test_size=0.2, random_state = 0)

In [44]:
from sklearn.ensemble import RandomForestClassifier
RF_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', RandomForestClassifier(n_estimators = 1000, max_depth = 5))
                             ])

In [45]:
#X_train,X_test,y_train,y_test = train_test_split(train_x, train_y,test_size=0.3, random_state = 0)
#clf = RF_pipeline.fit(X_train, y_train)
#importancia_de_features = RF_pipeline.steps[1][1].feature_importances_
#for feature in sorted(zip(importancia_de_features,features),reverse=True):
  #print(feature)

KeyboardInterrupt: 

In [ ]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [ ]:
my_pipeline.fit(train_x, train_y)

In [ ]:
preds = my_pipeline.predict(train_x)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(preds, train_y)

In [ ]:
preds_finales = my_pipeline.predict(prueba)

In [ ]:
res = pd.DataFrame(preds_finales, index=prueba.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)